### Chapter two - Reed-Frost dynamics, a template for stochastic epidemic models

#### Contents:
* [Assumptions and setup](#setup)
* [Probability of escape and infection](#escape)
* [Initial Conditions](#init)
* [Evolution](#evolution)
* [Reed Frost Dynamical System](#reedfrost)
* [Simulation](#simulation)
* [When p is small](#psmall)
* [Fitting the Reed-Frost to observations](#fit2data)
* [Using the computer to optimize](#computer)

#### Assumptions and setup <a class="anchor" id="setup"></a>

Suppose that there exists $N$ individuals who can exist in one of three disease states starting at time $0$ and ending at time $T$:
1. Susceptible - An individual is not infected at present but can be infected in the future.
2. Infected - An individual is infected at present and can infect other susceptible individuals. 
3. Removed - An individual is no longer susceptible, nor infected, and cannot again become infected. 

- Assume, for each time unit, that all $N$ individuals come into contact with one another, and that each interaction is independent. 
When a susceptible comes into contact with an infected, the susceptible individual has a probability $p$ of moving from the susceptible state to the infected state ($S \to I$). 

- Assume that if an individual is infected then they (1) are only infected for a single time unit and (2) after that time units ends they are removed. 

- Assume that the probability of infection should depend on the number of infected individuals in the population.

#### Probability of escape and infection <a class="anchor" id="escape"></a>

Consider a single susceptible individual. 
At time $t$, this susceptible individual will contact all $I$ infected individuals. 
At each contact between the susceptible individual and one of the infected individuals, there is a probabilty $p$ that the infected individual is successfull and infects this suscpetible individual.

If $p$ is the probability that an infected individual---often called can infector---can infect this suceptible then the probability that an infector does **not** infect this suscpetible is $1-p$.
Then the probability that this susceptible is not infected by the first member in $I$ is $(1-p)$ and the probability that this this susceptible is not infected by the first **and** second infector is 

$$
(1-p) \times (1-p)
$$

and so the probability that this suscpetible is not infected by all infectors in $I$ is 

$$
p(\text{escape}) = (1-p) \times (1-p)\times (1-p) \cdots \times (1-p) = (1-p)^{I}
$$
This is typically called the **esape probability**. 

If $(1-p)^{I}$ is the probability that no infector successfully infects this susceptible then the probability that at least one individual infects this suscpetible is 

$$
 p(\text{infection}) = 1 - \left[(1-p)^{I}\right]
$$






#### Initial conditions <a class="anchor" id="init"></a>

The above assumptions and conditions assume that the $N$ individuals who can exist in one of three disease states may move from one disease state to another *over time*.
Let us be explicit, and define the number of individuals in each disease state at time $0$, and define how the number of indivduals in each of the three disease states is expected to change over time. 

The **initial conditions** assigns a number of individuals to each disease state at time $0$, and is denoted 
\begin{align}
i_{0} &= i^{0}\\ 
s_{0} &= s^{0}\\
r_{0} &= r^{0}
\end{align}
where lowercase letters denote an observed or realized number of individuals in each disease state at time $0$ (the subscript) and lowercase letters with a superscript denote values between $0$ and $N$. 
For this model, the sum of the number of individuals in each disease state must equal $N$. 

#### Evolution<a class="anchor" id="evolution"></a>

Now that we have specified, at time $0$, the number of individuals who belong to the susceptible disease state ($s_{0}$), number of individuals who belong to the infected disease state ($i_{0}$), and number of individuals who belong to the removed disease state ($r_{0}$), we must decide the number of individuals at **the next time step**.

At time $t$, suppose that there exists $S_{t}$ susceptible individuals. 
A susceptible individual moves to the infected state with probability $1 - (1-p)^{I_{t}}$ (see above section on escape). 
Further, we assume that for each susceptible the probability of moving to the infected state is independent. 
If we define "success" as $1 - (1-p)^{I_{t}}$ and the number of trials as $S_{t}$ then the **number of infected individuals at time $t+1$ or $I_{t+1}$** has a Binomial distribution, 

\begin{align}
I_{t+1} &\sim \text{Bin}\left(S_{t}, \left[1 - (1-p)^{I_{t}}\right] \right).
\end{align}

Let the observed number of infected individuals at time $t+1$ equal $i_{t+1}$.
Then the number of susceptibles at the next time step is then equal to the number of susceptibles at time $t$ minus the number of susceptibles who became infected. 

\begin{align}
s_{t+1} &= s_{t} - i_{t+1} \\ 
\end{align}

Finally, we assume that at the end of each time unit all infected individuals at time $t$ move to the "removed" category. So then, 
\begin{align}
r_{t+1} &= r_{t}+ i_{t}
\end{align}





#### The Reed Frost Dynamical System<a class="anchor" id="reedfrost"></a>

The system of equations 
\begin{align}
I_{t+1} &\sim \text{Bin}\left(S_{t}, \left[1 - (1-p)^{I_{t}}\right] \right)\\
s_{t+1} &= s_{t} - i_{t+1} \\ 
r_{t+1} &= r_{t}+ i_{t}
\end{align}

along with initial conditions 
\begin{align}
(s^{0}, i^{0},r^{0})
\end{align}

define a system called **The Reed Frost Dynamical System**. 
We note that the captial $I_{t+1}$ indicates that the number of infectors at time $t+1$ is a random variable. 
After the number of infectors is determined, the number of susceptibles is automatically determined. 
However, before the number of infectors is selected the number of susceptibles is itself a random variable. 

One should be careful about when the number of susceptibles at time $t+1$ is a random variable.
Before the number of infectors is assigned the number of susceptibles  is a random variable. After the number of infectors is assigned then the number of susceptibles is not random. 

#### Simulation<a class="anchor" id="simulation"></a>

Below is a simulation of the above Reed Frost Dynamical System.
Presented is the number of infected individual at each time unit for a specified number of simulations. 
At this point, the code (in Python) is not important.

Instead, please select different values for $p$, the probability of infection; $i^{0} the initial number of infectors; and sims, the number of unique times to run this Reed Frost System. 

In [39]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def Reed_Frost_Dynamical_system(sims=10, T=15, N=1000, i0=1, p=0.02  ):
    def evolve(T,N,i0,p):
        import numpy as np 
        import matplotlib.pyplot as plt

        infectors    = [i0]
        removed      = [0]
        susceptibles = [N-i0]
        for t in range(T):
            i_tp1 = np.random.binomial(n=susceptibles[-1], p=1-(1-p)**( infectors[-1] ) , size=1  ) [0]
            s_tp1 = susceptibles[-1] - i_tp1
            r_tp1 = removed[-1] + infectors[-1]

            infectors.append(i_tp1)
            susceptibles.append(s_tp1)
            removed.append(r_tp1)
        infectors = np.array(infectors)

        return infectors

    fig, ax = plt.subplots()
    for _ in range(sims):
        infectors = evolve(T,N,i0,p)
        ax.plot(infectors)
    ax.set_ylim(0,75)
    ax.set_xlabel("Time units (t)",fontsize=10)
    ax.set_ylabel("Number of infectors (It)",fontsize=10)
    plt.show()
        
interact(Reed_Frost_Dynamical_system
         ,T=fixed(15)
         ,N=fixed(100)
         ,r0=fixed(0)
         ,i0=widgets.IntSlider(min=0, max=20, step=1, value=1) 
         ,p=(0,0.1,0.00125))

interactive(children=(IntSlider(value=10, description='sims', max=30, min=-10), IntSlider(value=1, description…

<function __main__.Reed_Frost_Dynamical_system(sims=10, T=15, N=1000, i0=1, p=0.02)>

#### Dynamics assumed at start of outbreak 





#### The ability to study epidemic chains 

Define an **epidemic chain** as the number of infected individuals over discrete time units. 
We will denote an epidemic chain as $i^{1}-i^{2}-i^{3}-i^{4}-i^{5}-\cdots$
where $i^{1}$ specifies the number of infected individuals within the first time unit, $i^{2}$ specifies the number of infected within the second time units and so on. 

*For example*, if we consider a household of 4 individuals then a feasible chain is 
1-2-1 and can be read "one individual was infected. This indiviudal infected two household members, and one of these two household members (not the original infected individual) infected the last household member who was still susceptible.". 


In [1]:
Relationship between lambda and 1-(1-p)^{I}
Susceptible with k contacts per unit time
proportion I infected
probability p of transmission
They make k*I contacts with infected
Prob of escape = (1-p)^{k*I}
Prob of infection = 1 - (1-p)^{k*I}
Approx Bi where B = klog(1-p)

SyntaxError: invalid syntax (<ipython-input-1-12d50507e955>, line 1)

In [ ]:
Fit Reed-Frost to data 
The Likelihood function and chain-binomial model
Building a likelihood function 
